In [17]:
# dependencies and setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# import API key

from config import gKey

In [18]:
# create data frame from CSV file

cities_df = pd.read_csv("Output_Data/cities.csv")

In [19]:
cities_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93


In [20]:
# configure gmaps

gmaps.configure(api_key = gKey)

In [21]:
# use lat lng as locations and add humidity as weight

locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [22]:
# add heat layer to map

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
cities_df.columns

Index(['City_ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat',
       'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [24]:
#create new data frame fitting my weather criteria - hot, humid, and a little breezy
ideal_df = cities_df.loc[(cities_df["Max Temp"] < 110) & 
                         (cities_df["Max Temp"] > 75) &
                         (cities_df["Humidity"] > 50) & 
                         (cities_df["Cloudiness"] < 10) & 
                         (cities_df["Wind Speed"] > 4) & 
                         (cities_df["Wind Speed"] < 20)]
ideal_df.dropna()
ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
157,157,charlestown,1,US,1558378784,64,41.38,-71.64,87.01,12.75
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
247,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
326,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
337,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41


In [25]:
# Store into variable named hotel_df & reset index for readability
hotel_df = ideal_df
hotel_df = hotel_df.reset_index().drop(columns="index")

In [26]:
# add a "hotel name" column to the data frame
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
2,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
4,157,charlestown,1,US,1558378784,64,41.38,-71.64,87.01,12.75,
5,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,
6,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17,
7,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
8,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,
9,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,


In [27]:
# set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gKey,
}

# iterate through 
for i, r in hotel_df.iterrows():
#get lat, lng from data frame    
    lat = r["Lat"]
    lng = r["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request
    names = requests.get(base_url, params=params)
    names = names.json()
    
    #save hotel name to hotels dataframe
    try:
        hotel_df.loc[i, "Hotel Name"] = names["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info... skipping.")
hotel_df        

Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.
Missing info... skipping.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
2,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
4,157,charlestown,1,US,1558378784,64,41.38,-71.64,87.01,12.75,
5,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17,
6,247,puerto escondido,5,MX,1558378824,66,15.86,-97.07,86.00,9.17,
7,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,
8,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,
9,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))